In [1]:
import numpy as np

In [2]:
np.__version__

'1.19.5'

In [15]:
import numpy as np
import re

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import os

from konlpy.tag import Kkma
from konlpy.tag import Okt
okt=Okt()
text=pd.read_csv('223.csv')

###############파일 우리 파일로 바꿔야함 (원본 + 번역본 함께 활용)


In [16]:
#유니코드를 아스키로
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')

#문장부호를 분리하려고
def preprocess_sentence(sent):
    sent = unicode_to_ascii(sent)
    sent = re.sub(r"([?.!,¿])", r" \1", sent)
    sent = re.sub(r"([^a-zA-Z!.?])", r" ", sent)
    sent = re.sub(r"\s+", " ", sent)
    return sent 

#읽어들인 데이터를 전처리
#영어는 encoder_input, 프랑스어는 decoder_input과 decoder_target
#decoder_input은 teacher forcing을 위해, decoder_target은 테스트를 위해
def load_preprocessed_data():
    encoder_input, decoder_input, decoder_target = [], [], []
    
    ###################파일 우리 데이터로 변경.
    
    pd.read_csv("223.csv", "r",encoding='UTF8') as lines:
        for i, line in enumerate(lines):
            src_line, tar_line, _ = line.strip().split('\t')
            src_line_input = [w for w in preprocess_sentence(src_line).split()]

            tar_line = preprocess_sentence(tar_line)
            tar_line_input = [w for w in ("<sos> " + tar_line).split()]
            tar_line_target = [w for w in (tar_line + " <eos>").split()]

            encoder_input.append(src_line_input)
            decoder_input.append(tar_line_input)
            decoder_target.append(tar_line_target)

            if i == num_samples - 1 :
                break

    return encoder_input, decoder_input, decoder_target   
    
#전처리 데이터 테스트
sents_en_in, sents_fra_in, sents_fra_out = load_preprocessed_data()
print(sents_en_in[:5])
print(sents_fra_in[:5])
print(sents_fra_out[:5])

SyntaxError: invalid syntax (<ipython-input-16-3ec472d827c0>, line 22)

In [5]:
####################한글 토큰화로 변경


#소스(영어) 문장 토큰화
tokenizer_en = Tokenizer(filters="", lower=False)
tokenizer_en.fit_on_texts(sents_en_in)
encoder_input = tokenizer_en.texts_to_sequences(sents_en_in)

#타겟(프랑스) 문장 토큰화
tokenizer_fra = Tokenizer(filters="", lower=False)
tokenizer_fra.fit_on_texts(sents_fra_in)
tokenizer_fra.fit_on_texts(sents_fra_out)
decoder_input = tokenizer_fra.texts_to_sequences(sents_fra_in)
decoder_target = tokenizer_fra.texts_to_sequences(sents_fra_out)

#토큰화 된 문장 패딩
encoder_input = pad_sequences(encoder_input, padding="post")
decoder_input = pad_sequences(decoder_input, padding="post")
decoder_target = pad_sequences(decoder_target, padding="post")

#사전 사이즈
src_vocab_size = len(tokenizer_en.word_index) + 1
tar_vocab_size = len(tokenizer_fra.word_index) + 1

#추후 활용을 위하 사전의 인덱스와 단어를 리스트로 저장
src_to_index = tokenizer_en.word_index
index_to_src = tokenizer_en.index_word
tar_to_index = tokenizer_fra.word_index
index_to_tar = tokenizer_fra.index_word

#학습이 더 고르게 되도록 데이터를 셔플
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

#train, eval 데이터로 train 데이터를 분리 
n_of_val = int(33000*0.1)
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]
encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [6]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

# 은닉층의 차원은 50개로 
latent_dim = 50

# 토큰화된 영어로 인코더 구성
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(src_vocab_size, latent_dim)(encoder_inputs)
# masking의 이유는 padding된 '0'는 학습에 이용되지 않도록
enc_masking = Masking(mask_value=0.0)(enc_emb)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking)
encoder_states = [state_h, state_c]

# 토큰화된 프랑스어로 디코더 구성
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(tar_vocab_size, latent_dim) # 임베딩 층
dec_emb = dec_emb_layer(decoder_inputs) # 패딩 0은 연산에서 제외
dec_masking = Masking(mask_value=0.0)(dec_emb)

# 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_sequences는 True
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True) 

# 인코더의 은닉 상태를 초기 은닉 상태(initial_state)로 사용
decoder_outputs, _, _ = decoder_lstm(dec_masking,
                                     initial_state=encoder_states)
                                     
# softmax로 decoder의 출력 구성
decoder_dense = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])

model.fit(x = [encoder_input_train, decoder_input_train], y = decoder_target_train, \
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size = 128, epochs = 5)

Epoch 1/5
233/233 [==============================] - 63s 214ms/step - loss: 3.2098 - acc: 0.6185 - val_loss: 1.9191 - val_acc: 0.6539
Epoch 2/5
233/233 [==============================] - 47s 201ms/step - loss: 1.7089 - acc: 0.7363 - val_loss: 1.6001 - val_acc: 0.7475
Epoch 3/5
233/233 [==============================] - 49s 210ms/step - loss: 1.4939 - acc: 0.7572 - val_loss: 1.4707 - val_acc: 0.7644
Epoch 4/5
233/233 [==============================] - 48s 204ms/step - loss: 1.3898 - acc: 0.7761 - val_loss: 1.3813 - val_acc: 0.7799
Epoch 5/5
233/233 [==============================] - 50s 213ms/step - loss: 1.3155 - acc: 0.7866 - val_loss: 1.3227 - val_acc: 0.7897


In [7]:
# 인코더 모델은 구성한 그대로 사용
encoder_model = Model(encoder_inputs, encoder_states)

# 디코더 모델
# 이전 시점의 상태를 보관할 텐서
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 훈련 때 사용했던 임베딩 층을 재사용
dec_emb2= dec_emb_layer(decoder_inputs)

# 다음 단어 예측을 위해 이전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

# 모든 시점에 대해서 단어 예측
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)
 

#encoder, decoder 모델을 별도로 구성해 둔다. 참고로 Keras의 Model 함수의 의미는 아래와 같다.

 

from keras.models import Model
from keras.layers import Input, Dense

a = Input(shape=(32,))
b = Dense(32)(a)
model = Model(inputs=a, outputs=b)

# input 'a'와, output 'b'를 구성하기 위한 모든 layer를 자동으로 구성함

In [8]:
# 실제 테스트 함수
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 정수 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = tar_to_index['<sos>']

    stop_condition = False
    decoded_sentence = ''

    # stop_condition이 True가 될 때까지 루프 반복
    # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정합니다.
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 단어로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]

         # 현재 시점의 예측 단어를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char

        # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
        if (sampled_char == '<eos>' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [9]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2src(input_seq):
    temp=''
    for i in input_seq:
        if(i!=0):
            temp = temp + index_to_src[i]+' '
    return temp

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2tar(input_seq):
    temp=''
    for i in input_seq:
        if((i!=0 and i!=tar_to_index['<sos>']) and i!=tar_to_index['<eos>']):
            temp = temp + index_to_tar[i] + ' '
    return temp
    
# 테스트 모듈
for seq_index in [3,50,100,300,1001]:
    input_seq = encoder_input_train[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)

    print("원문 : ",seq2src(encoder_input_train[seq_index]))
    print("번역문 :",seq2tar(decoder_input_train[seq_index]))
    print("예측문 :",decoded_sentence[:-5])
    print("\n")


원문 :  Watch us . 
번역문 : Regarde nous ! 
예측문 :  Soyez la ! 


원문 :  What would happen ? 
번역문 : Que se passerait il ? 
예측문 :  Est ce vous ? 


원문 :  I said maybe . 
번역문 : J ai dit peut etre . 
예측문 :  Je suis suis . 


원문 :  Excuse me ? 
번역문 : Je vous demande pardon ? 
예측문 :  Est ce vous ? 


원문 :  I m pretty busy . 
번역문 : Je suis plutot occupee . 
예측문 :  Je suis suis . 




In [10]:
from konlpy.tag import Kkma
from konlpy.tag import Okt

In [12]:
import nltk

In [ ]:
#소스(영어) 문장 토큰화
tokenizer_en = Tokenizer(filters="", lower=False)
tokenizer_en.fit_on_texts(sents_en_in)
encoder_input = tokenizer_en.texts_to_sequences(sents_en_in)

#타겟(프랑스) 문장 토큰화



tokenizer_fra = Tokenizer(filters="", lower=False)
tokenizer_fra.fit_on_texts(sents_fra_in)
tokenizer_fra.fit_on_texts(sents_fra_out)
decoder_input = tokenizer_fra.texts_to_sequences(sents_fra_in)
decoder_target = tokenizer_fra.texts_to_sequences(sents_fra_out)

#토큰화 된 문장 패딩
encoder_input = pad_sequences(encoder_input, padding="post")
decoder_input = pad_sequences(decoder_input, padding="post")
decoder_target = pad_sequences(decoder_target, padding="post")

#사전 사이즈
src_vocab_size = len(tokenizer_en.word_index) + 1
tar_vocab_size = len(tokenizer_fra.word_index) + 1

#추후 활용을 위하 사전의 인덱스와 단어를 리스트로 저장
src_to_index = tokenizer_en.word_index
index_to_src = tokenizer_en.index_word
tar_to_index = tokenizer_fra.word_index
index_to_tar = tokenizer_fra.index_word

#학습이 더 고르게 되도록 데이터를 셔플
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

#train, eval 데이터로 train 데이터를 분리 
n_of_val = int(33000*0.1)
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]
encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [13]:
okt=Okt()
text=pd.read_csv('223.csv')

In [ ]:
okt